In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import os
from torchvision.transforms import v2
import torch
import shutil
from PIL import Image
from ultralytics import YOLO
import cv2
import pandas as pd
from PIL import Image
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from IPython.display import clear_output

# Local dep
project_dir = '/data/konrad/workspace'
sys.path.insert(0, project_dir)

from helpers.datasets import CalfCenterFaceDataset
from helpers.helpers import get_indices, uniform_sample_with_values, load_face_data

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [2]:
def plot_bbox_gallery(image_df, n_cols=5):
    n_images = image_df.shape[0]
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    for i, row in image_df.iterrows():
        img = Image.open(row["path"])
        # plt.subplot(n_rows, n_cols, i + 1)
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        # ax = plt.gca()
        score = row["conf"]
        box_x = row["box_x"]
        box_y = row["box_y"]
        box_width = row["box_width"]
        box_height = row["box_height"]
        box_x = box_x - (box_width / 2)
        box_y = box_y - (box_height / 2)
        rect = plt.Rectangle((box_x, box_y), box_width, box_height, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(box_x, box_y - 10, f' {score:.2f}', color='red', fontsize=12, backgroundcolor='white')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()


def apply_transformations(image, bbox, transforms):
    transformed = transforms(image=image, bboxes=[bbox], category_ids=[0])
    
    transformed_image = transformed['image']
    transformed_bbox = transformed['bboxes'][0]
    
    return transformed_image, transformed_bbox
    
def df_to_yolo(df, root_dir, transforms = None, num_gen=3, img_src_dir = None):
    
    img_path = root_dir + "/" + "images"
    if not os.path.exists(img_path):
        os.makedirs(img_path)

    lbl_path = root_dir + "/" + "labels"
    if not os.path.exists(lbl_path):
        os.makedirs(lbl_path)

    for index, row in df.iterrows():
        # Extract the image name
        image_name = row['path']

        img_src_dir = row['img_dir']
        
        class_idx = row['type']

        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']

        bbox = [xmin, ymin, xmax, ymax]
        
        if transforms is None:

            shutil.copy(os.path.join(img_src_dir, image_name), os.path.join(img_path, image_name))
            filename = os.path.splitext(image_name)[0]

            # Open the image file
            with Image.open(os.path.join(img_src_dir, image_name)) as img:
            # Get the width and height
                imgWidth, imgHeight = img.size

            center_x = (xmin + xmax) / 2.0
            center_y = (ymin + ymax) / 2.0
    
            width = xmax - xmin
            height = ymax - ymin
    
            norm_center_x = center_x / imgWidth
            norm_center_y = center_y / imgHeight
            
            norm_width = width / imgWidth
            norm_height = height / imgHeight
            
            box_annotation = ' '.join([str(class_idx), str(norm_center_x), str(norm_center_y), str(norm_width), str(norm_height)])+'\n'
    
            label_filename = lbl_path + '/' + filename + ".txt"
            anno_f = open(label_filename, 'w')
            anno_f.writelines(box_annotation)
            anno_f.close()

            continue

        original_image_path = os.path.join(img_src_dir, image_name)
        img = cv2.imread(original_image_path)
        imgHeight, imgWidth, _ = img.shape
        
        for i in range(num_gen):  # Apply transformations 5 times
            transformed_image, transformed_bbox = apply_transformations(img, bbox, transforms)
            
            center_x = (transformed_bbox[0] + transformed_bbox[2]) / 2.0
            center_y = (transformed_bbox[1] + transformed_bbox[3]) / 2.0

            width = transformed_bbox[2] - transformed_bbox[0]
            height = transformed_bbox[3] - transformed_bbox[1]
            
            norm_center_x = center_x / imgWidth
            norm_center_y = center_y / imgHeight
            
            norm_width = width / imgWidth
            norm_height = height / imgHeight
            
            box_annotation = ' '.join([str(class_idx), str(norm_center_x), str(norm_center_y), str(norm_width), str(norm_height)]) + '\n'
            
            transformed_image_name = f"{os.path.splitext(image_name)[0]}_transformed_{i}.jpg"
            cv2.imwrite(os.path.join(img_path, transformed_image_name), transformed_image)
            
            label_filename = os.path.join(lbl_path, f"{os.path.splitext(transformed_image_name)[0]}.txt")
            with open(label_filename, 'w') as anno_f:
                anno_f.writelines(box_annotation)

    print("Done !")


def delete_dir_if_exists(dir_path):
    if os.path.exists(dir_path):
        try:
            shutil.rmtree(dir_path)
            print(f"Directory '{dir_path}' deleted")
        except OSError as e:
            print(f"Error deleting directory '{dir_path}': {e}")

In [3]:
ROOT_DIR = "/data/konrad/workspace"
DATA_FILE = ROOT_DIR + '/datasets/CompleteDatasetNormalFace/Face_annotations.csv'
IMAGE_DIR = ROOT_DIR + "/datasets/CompleteDatasetNormalFace"

face_df, labels, label2id, id2label = load_face_data(DATA_FILE, IMAGE_DIR)
face_df["img_dir"] = IMAGE_DIR
face_df["type"] = 0

mix_df = pd.concat([face_df], ignore_index=True)

filter_values = {
    'calf': ['6842', '6436', '6864'],  # Specify the values you want to include
}

valid_df = mix_df[mix_df['calf'].isin(filter_values['calf'])]
train_df = mix_df[~ mix_df['calf'].isin(filter_values['calf'])]

IMAGE_DIR = ROOT_DIR + "/datasets/calfs_face"

In [4]:
num_exp = 10
max_perfo = 0 
records = []
base_epoch = 30
num_gen = 5

for _ in range(num_exp):

    delete_dir_if_exists(IMAGE_DIR)

    transforms = A.Compose([
        A.Rotate(limit=(10, 20), p=.7),
        A.OneOf([
            A.GaussianBlur(blur_limit=(3,9), p=.7),
            A.MedianBlur(blur_limit=7, p=.7),
        ], p=.6),
        A.Sharpen(p=.7),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))
    
    df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
    df_to_yolo(valid_df, IMAGE_DIR + '/test', img_src_dir = IMAGE_DIR)

    # Load a model
    model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
    
    results = model.train(data="yolo-face.yml", epochs=base_epoch, project=ROOT_DIR + "/training_log", name="yolo_face", cfg="config.yml")
    model.export()
    clear_output()
    
    metrics = results.results_dict
    metrics["save_dir"] = str(results.save_dir)
    metrics["try"] = _
    records.append(metrics)
    
    last_best_perfo = 0
    epochs = base_epoch + 10
    
    if max_perfo < results.results_dict["metrics/mAP50-95(B)"]:
        
        max_perfo = results.results_dict["metrics/mAP50-95(B)"]
        for __ in range(10):        
            # Load a model
            model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
            
            last_results = model.train(data="yolo-face.yml", epochs=epochs, project=ROOT_DIR + "/training_log", name="yolo_face", cfg="config.yml")
            model.export()
            clear_output()
    
            metrics = last_results.results_dict
            metrics["save_dir"] = str(last_results.save_dir)
            metrics["try"] = _
            records.append(metrics)
            
            if last_best_perfo > last_results.results_dict["metrics/mAP50-95(B)"]:
                break
    
            epochs+=10
            last_best_perfo = last_results.results_dict["metrics/mAP50-95(B)"]


        max_perfo = max(max_perfo, last_best_perfo)
        
    
records = pd.DataFrame(records)
records

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),fitness,save_dir,try
0,0.826476,0.555556,0.699150,0.459030,0.483042,/data/konrad/workspace/training_log/yolo_face11,0
1,0.991248,0.666667,0.903756,0.454722,0.499626,/data/konrad/workspace/training_log/yolo_face12,0
2,0.758097,0.699054,0.762188,0.424689,0.458439,/data/konrad/workspace/training_log/yolo_face13,0
3,0.828499,0.555556,0.640673,0.477700,0.493998,/data/konrad/workspace/training_log/yolo_face14,1
4,1.000000,0.769589,0.891786,0.502220,0.541177,/data/konrad/workspace/training_log/yolo_face15,1
5,0.828900,0.555556,0.685414,0.485180,0.505204,/data/konrad/workspace/training_log/yolo_face16,1
6,0.828499,0.555556,0.640673,0.477700,0.493998,/data/konrad/workspace/training_log/yolo_face17,2
7,0.828499,0.555556,0.640673,0.477700,0.493998,/data/konrad/workspace/training_log/yolo_face18,3
8,0.828499,0.555556,0.640673,0.477700,0.493998,/data/konrad/workspace/training_log/yolo_face19,4
9,0.828499,0.555556,0.640673,0.477700,0.493998,/data/konrad/workspace/training_log/yolo_face20,5


In [5]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
    # A.OneOf([
    #     A.ChannelShuffle(p=.7),
    #     A.RGBShift(p=.7),
    #     A.InvertImg(p=.5),
    #     A.ToGray(p=.8),
    # ], p=.6),
    # A.OneOf([
    #     A.Sharpen(p=.7),
    #     A.ColorJitter(brightness=0.2, contrast=0.5, saturation=0.5, hue=0.2, p=.7)
    # ], p=.6)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))
num_gen = 5

delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/test', img_src_dir = IMAGE_DIR)

Directory '/data/konrad/workspace/datasets/calfs_face' deleted
Done !
Done !


In [6]:
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

results = model.train(data="yolo-face.yml", epochs=5, project=ROOT_DIR + "/training_log", name="yolo_face", cfg="config.yml")

model.export()

New https://pypi.org/project/ultralytics/8.2.74 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo-face.yml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_face25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

train: Scanning /data/konrad/workspace/datasets/calfs_face/train/labels... 890 images, 0 backgrounds, 0 corrupt: 100%|██

train: New cache created: /data/konrad/workspace/datasets/calfs_face/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /data/konrad/workspace/datasets/calfs_face/test/labels... 9 images, 0 backgrounds, 0 corrupt: 100%|███████

val: New cache created: /data/konrad/workspace/datasets/calfs_face/test/labels.cache


...

                   all          9          9          1      0.666      0.811       0.48



5 epochs completed in 0.039 hours.
Optimizer stripped from /data/konrad/workspace/training_log/yolo_face25/weights/last.pt, 6.2MB
Optimizer stripped from /data/konrad/workspace/training_log/yolo_face25/weights/best.pt, 6.2MB

Validating /data/konrad/workspace/training_log/yolo_face25/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00


                   all          9          9          1      0.666      0.811       0.48
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_face25
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_face25/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 0.8s, saved as '/data/konrad/workspace/training_log/yolo_face25/weights/best.torchscript' (11.9 MB)

Export complete (2.1s)
Results saved to /data/konrad/workspace/training_log/yolo_face25/weights
Predict:         yolo predict task=detect model=/data/konrad/workspace/training_log/yolo_face25/weights/best.torchscript 

'/data/konrad/workspace/training_log/yolo_face25/weights/best.torchscript'